In [296]:
import import_ipynb
from Task_2 import hamming_distance as HD
from lib import bn256
import random
import numpy as np
import copy

In [297]:
v1, v2 = (np.random.choice([-1, 1], 16) for _ in range(2))
v1, v2

(array([ 1, -1,  1,  1, -1,  1,  1,  1,  1,  1,  1,  1, -1, -1, -1, -1]),
 array([-1, -1,  1, -1,  1, -1, -1,  1, -1,  1, -1, -1, -1, -1, -1,  1]))

In [298]:
G1 = bn256.curve_G
G2 = bn256.twist_G

In [299]:
v1points = [G2.scalar_mul(bn256.order + i) for i in v1]
v2points  = [G1.scalar_mul(bn256.order + i) for i in v2]

In [300]:
#check
v2points[0].add(v2points[1])
v1points[3].add(v1points[2])

((59927578126862636086832814129077742734260689452696771010895412296143667746211,54952930061809850496692595958158609574339126978755622048249382632547867406485), (16688816548089249544807710431443648737000867162658939349813935924644263586693,63452769682808524161861388928032618183185319239119072364816295620689227591133))

In [301]:
v1points

[((21167961636542580255011770066570541300993051739349375019639421053990175267184,64746500191241794695844075326670126197795977525365406531717464316923369116492), (20666913350058776956210519119118544732556678129809273996262322366050359951122,17778617556404439934652658462602675281523610326338642107814333856843981424549)),
 ((21167961636542580255011770066570541300993051739349375019639421053990175267184,64746500191241794695844075326670126197795977525365406531717464316923369116492), (44333636345587826776585919623241361010268679977813729575614822660813824120661,47221932139242163798143780279757230461301747781284361464062811170020202647234)),
 ((21167961636542580255011770066570541300993051739349375019639421053990175267184,64746500191241794695844075326670126197795977525365406531717464316923369116492), (20666913350058776956210519119118544732556678129809273996262322366050359951122,17778617556404439934652658462602675281523610326338642107814333856843981424549)),
 ((2116796163654258025501177006657054

In [302]:
v2points

[(1, 2),
 (1, 2),
 (1, 65000549695646603732796438742359905742825358107623003571877145026864184071781),
 (1, 2),
 (1, 65000549695646603732796438742359905742825358107623003571877145026864184071781),
 (1, 2),
 (1, 2),
 (1, 65000549695646603732796438742359905742825358107623003571877145026864184071781),
 (1, 2),
 (1, 65000549695646603732796438742359905742825358107623003571877145026864184071781),
 (1, 2),
 (1, 2),
 (1, 2),
 (1, 2),
 (1, 2),
 (1, 65000549695646603732796438742359905742825358107623003571877145026864184071781)]

In [303]:
h1, h2 = (random.randrange(2, bn256.order) for _ in range(2))
H1, H2 = G1.scalar_mul(h1), G2.scalar_mul(h2)
H1, H2, h1, h2

((37920139704601244796461081380993549109597733223549191971084964941220261951039, 21827865559301339992164828902857479197825161097289622186269946674681850042339),
 ((14508919276704119018322976448853401565949466114646173071101982964485971273564,62645819862036814653073110472702196063511842964441435999506396094483994219034), (43369924552598677067203167604021685291418033033311140812608982999151471956693,41112061595939159664837588676314286409598771798622361965358295269100693680895)),
 58078700230173686313805395047221678411439302531479740994780238800652535364518,
 44630902988099026950142644118952898055819778583435612307459065356532318004016)

In [304]:
def CIPE_setup():
    s, t = ([random.randrange(2, bn256.order) for i in range(16)] for _ in range(2))
    u, v = ([random.randrange(2, bn256.order) for i in range(18)] for _ in range(2))
    gen1_h = [G1.scalar_mul(s[i]).add(H1.scalar_mul(s[i])) for i in range(16)]
    gen2_h = [G2.scalar_mul(u[i]).add(H2.scalar_mul(v[i])) for i in range(18)]
    return s, t, u, v, gen1_h, gen2_h

In [305]:
s, t, u, v, gen1_h, gen2_h = CIPE_setup()
s, t, u, v, gen1_h, gen2_h

([11284764821232128005370490483764352420417916951655660790526591822576083276074,
  39233969326749794378495076080448108272075577494815901095479503273517543687494,
  14827640533089733774119749601055307347477996352661544194145806283221017377485,
  36917552890004787232032106019254397747795622756489888852101570255144542943537,
  4677716074048623541011629360796287612710167559596866116907512187493116332199,
  44568343646791028525232766480616464426460800518117294869534008141163186945646,
  8375447691745356712704174093641270500660112672167378094278392631921247152438,
  56107868481204060341004342359830603381342522216404996351143262071074387258810,
  4355193466288661292579943975940058697539646411803761311763035128655671500124,
  51499479396172267688182482678006025324158658796412747464216837290834244037128,
  36677348192489570600308109460010739790241959952476388632979431413833970363423,
  19264659115612239628271718614444045830454055544546184546869478242064448331451,
  25104690505173387707659753602

In [306]:
def Registration(G2, s, t, v1points):
    r0 = random.randrange(2, bn256.order)
    reg_template = [None for _ in range(20)]
    reg_template[0] = G2.scalar_mul(r0)
    reg_template[1] = H2.scalar_mul(r0)
    for i in range(18):
        reg_template[i+2] = gen2_h[i].scalar_mul(r0)
    for i in range(16):
        vs = v1points[i].scalar_mul(s[i])
        vt = v1points[i].scalar_mul(t[i])
        vs.negate()
        vt.negate()
        reg_template[2] = reg_template[2].add(vs)
        reg_template[3] = reg_template[3].add(vt)
    sum_v = v1points[0]
    for i in range(1, 16):
        sum_v = vv.add(v1points[i])
    for i in range(16):
        reg_template[i+4] = reg_template[i+4].add(sum_v)
    
    return reg_template

In [307]:
def Authentication(G2, u, v, v2points):
    r0 = random.randrange(2, bn256.order)
    auth_template = [None for _ in range(20)]
    auth_template[0] = G1.scalar_mul(0)
    auth_template[1] = G1.scalar_mul(0)
    auth_template[2] = G1.scalar_mul(r0)
    auth_template[3] = H1.scalar_mul(r0)
    for i in range(16):
        auth_template[i+4] = gen1_h[i].scalar_mul(r0).add(v2points[i])
    for i in range(18):
        vu = auth_template[i+2].scalar_mul(u[i])
        vv = auth_template[i+2].scalar_mul(u[i])
        vu.y = vu.y.additive_inverse()
        vv.y = vv.y.additive_inverse()
        auth_template[0] = auth_template[0].add(vu)
        auth_template[1] = auth_template[1].add(vv)
    return auth_template

In [308]:
reg_template = Registration(G2, s, t, v1points)
auth_template = Authentication(G2, u, v, v2points)

In [309]:
table = [1]
oa = bn256.optimal_ate(G2, G1)
for i in range(1,16):
    oa = oa.exp(i)
    table.append(oa)

In [310]:
table

[1,
 (((21196725492326442667627602847448291017588785091108454726553939930507871042194,43377286344385619261366633786771510260308325523330699049673437042931316243080),(6519643025219345838017222895845262421933106748812739147102715685220942156402,49167428805016148020211692433963935169227827998815966523759923767649702416286),(20859153989312102139134197789193625833054352751342763983124765227328906753159,54487609867103086000679472636440811221594782878893833473117980998935745956453)),((39332187218762343173097683922586548248512461497033773500717078587710862589062,53094021597187285527531149248961924798783924165048746910730430368152798446315),(30733062774817315099333283633560206070773769397463591953591634872711994123707,13560812839206871407210482171294630929511117297628119163695762035076749250365),(57080396271372976186541856910255122494067079575964633601781325258931774013251,60034081832369659851990276215766505463071420460830584339216728009871686767851))),
 (((5299302969398864688583225092139732038

In [311]:
#e = bn256.optimal_ate(reg_template[0], auth_template[0])
#for i in range(1,20):
 #   e = bn256.gfp_12.mul(e, bn256.optimal_ate(reg_template[i], auth_template[i]))